# 🎓 Building a Sentiment Classifier with Logistic Regression

In this notebook, we will build a **Logistic Regression** classifier to predict the sentiment (positive or negative) of customer reviews for women's clothing from an e-commerce website. 

For this task, we will use the **LogisticRegression** class from the scikit-learn library.


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
file_path = 'womens_clothing_ecommerce_reviews.csv'
df = pd.read_csv(file_path)
print("Successfully loaded the dataset.")
print("Dataset preview:")
print(df.head())

Successfully loaded the dataset.
Dataset preview:
                                         Review Text  sentiment
0  Absolutely wonderful - silky and sexy and comf...          1
1  Love this dress!  it's sooo pretty.  i happene...          1
2  I love, love, love this jumpsuit. it's fun, fl...          1
3  This shirt is very flattering to all due to th...          1
4  I love tracy reese dresses, but this one is no...         -1


In [5]:
# Get some basic information about the dataset
print("\nDataset Information:")
df.info()


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19818 entries, 0 to 19817
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review Text  19818 non-null  object
 1   sentiment    19818 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 309.8+ KB


In [6]:
# --- Step 2: Split Data into Training and Testing Sets ---
# It's crucial to test our model on data it has never seen before.
# We'll use 80% of the data for training and 20% for testing.
X = df['Review Text']
y = df['sentiment']

# 'stratify=y' ensures that the proportion of positive and negative reviews is the same in both your training set and your testing set.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"\nData split into {len(X_train)} training samples and {len(X_test)} testing samples.")


Data split into 15854 training samples and 3964 testing samples.


In [7]:
# --- Step 3: Feature Engineering with Bag-of-Words ---
# Here, we convert the text reviews into numerical feature vectors.
# Each feature is a count of how many times a word appears in a review.
print("\nConverting text to numerical features using Bag-of-Words...")

# Initialize the vectorizer. `stop_words='english'` removes common
# English words like 'the', 'a', 'is', which don't carry much sentiment.
vectorizer = CountVectorizer(stop_words='english')

# Fit the vectorizer on the TRAINING data and transform it into a matrix
X_train_bow = vectorizer.fit_transform(X_train)

print(f"X_train_bow Shape:\n {X_train_bow.shape}")

print(f"Unique words (features): {vectorizer.get_feature_names_out()}")

# Output is a sparse matrix representation, where most entries are zero and only non-zero values are stored
print(f"X_train_bow:\n {X_train_bow}")

# ONLY transform the TESTING data using the already-fitted vectorizer
X_test_bow = vectorizer.transform(X_test)

print("✅ Text successfully converted to feature vectors.")


Converting text to numerical features using Bag-of-Words...
X_train_bow Shape:
 (15854, 11897)
Unique words (features): ['00' '000' '00p' ... 'zooming' 'zuma' 'ã¼ber']
X_train_bow:
 <Compressed Sparse Row sparse matrix of dtype 'int64'
	with 373991 stored elements and shape (15854, 11897)>
  Coords	Values
  (0, 10214)	1
  (0, 2396)	1
  (0, 4805)	1
  (0, 4210)	1
  (0, 3835)	1
  (0, 9289)	1
  (0, 3680)	1
  (0, 4669)	1
  (0, 6980)	1
  (0, 10773)	1
  (0, 7248)	1
  (0, 7409)	1
  (1, 9289)	1
  (1, 2866)	1
  (1, 3454)	1
  (1, 4206)	1
  (1, 1781)	1
  (1, 6221)	1
  (1, 6711)	1
  (1, 8861)	1
  (1, 9671)	1
  (1, 9424)	1
  (1, 11)	1
  (2, 7409)	1
  (2, 3454)	3
  :	:
  (15853, 4805)	1
  (15853, 7596)	1
  (15853, 6257)	1
  (15853, 2390)	1
  (15853, 11495)	1
  (15853, 1602)	1
  (15853, 8053)	1
  (15853, 4742)	1
  (15853, 8467)	1
  (15853, 7829)	1
  (15853, 9466)	1
  (15853, 10594)	1
  (15853, 11408)	1
  (15853, 11414)	1
  (15853, 8195)	1
  (15853, 584)	1
  (15853, 11438)	1
  (15853, 5070)	1
  (15853

In [8]:
# --- Step 4: Train the Linear Classifier ---
# We'll use Logistic Regression, a reliable linear model for classification.
print("\nTraining the Logistic Regression model...")

# Initialize the model
# max_iter is increased to ensure the model has enough time to find the best weights
# solver: Algorithm for optimization (default: 'lbfgs'). Options: 'lbfgs', 'liblinear', 'saga', 'newton-cg', 'sag'
model = LogisticRegression(max_iter=5000, solver='saga', random_state=42)

# Train the model on our Bag-of-Words training data
model.fit(X_train_bow, y_train)

print("✅ Model training complete!")


Training the Logistic Regression model...
✅ Model training complete!


In [9]:
# --- Step 5: Evaluate the Model's Performance ---
# Let's see how accurately our model predicts sentiment on the unseen test data.
print("\nEvaluating model performance on the test set...")

# Make predictions on the test data
y_pred = model.predict(X_test_bow)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"📈 Model Accuracy: {accuracy:.4f} ({accuracy:.2%})")


Evaluating model performance on the test set...
📈 Model Accuracy: 0.9299 (92.99%)


In [10]:
#to save the model
import joblib


joblib.dump(model, 'model.joblib')

['model.joblib']

In [12]:
#save the vectorizer file
joblib.dump(vectorizer, 'vectorizer.joblib')

['vectorizer.joblib']

In [11]:
# --- Step 6: Predict Sentiment on New Reviews ---
# This is the fun part! Let's use our fine-tuned model on brand new text.
print("\n--- Making Predictions on New Reviews ---")

new_reviews = [
    "This dress is absolutely beautiful and fits perfectly!",
    "The material felt cheap and it was not what I expected.",
    "It's an okay product, not great but not terrible either.",
    "I am so disappointed with this purchase, I will be returning it.",
    "I want to buy this again and again!",
]

# 1. Transform the new reviews into the Bag-of-Words format
new_reviews_bow = vectorizer.transform(new_reviews)
print("new_reviews_bow shape:", new_reviews_bow.shape)

# 2. Predict using our trained model
new_predictions = model.predict(new_reviews_bow)

print(type(new_predictions))
print(new_predictions)


for i in range(len(new_reviews)):
    print(f"Review: {new_reviews[i]}")
    print(f"Predicted Sentiment: {new_predictions[i]}\n")


--- Making Predictions on New Reviews ---
new_reviews_bow shape: (5, 11897)
<class 'numpy.ndarray'>
[ 1 -1 -1 -1  1]
Review: This dress is absolutely beautiful and fits perfectly!
Predicted Sentiment: 1

Review: The material felt cheap and it was not what I expected.
Predicted Sentiment: -1

Review: It's an okay product, not great but not terrible either.
Predicted Sentiment: -1

Review: I am so disappointed with this purchase, I will be returning it.
Predicted Sentiment: -1

Review: I want to buy this again and again!
Predicted Sentiment: 1

